In [1]:
cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


# BW

In [2]:
import numpy as np
import cv2

net = cv2.dnn.readNetFromCaffe('model/colorization_deploy_v2.prototxt','model/colorization_release_v2.caffemodel')
pts = np.load('model/pts_in_hull.npy')

class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

def colorize(im_name):
	image = cv2.imread(im_name)
	scaled = image.astype("float32") / 255.0
	lab = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)
	resized = cv2.resize(lab, (224, 224))
	L = cv2.split(resized)[0]
	L -= 50
	net.setInput(cv2.dnn.blobFromImage(L))
	ab = net.forward()[0, :, :, :].transpose((1, 2, 0))
	ab = cv2.resize(ab, (image.shape[1], image.shape[0]))
	L = cv2.split(lab)[0]
	colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)
	colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
	colorized = np.clip(colorized, 0, 1)
	colorized = (255 * colorized).astype("uint8")
	cv2.imwrite(im_name, colorized)
	cv2.waitKey(0)

# Flask

Server

In [3]:
!pip install flask-ngrok

In [ ]:
import os, requests, socket
import numpy as np
import tensorflow as tf
from flask import Flask, request, send_file
print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))

app = Flask(__name__)

from flask_ngrok import run_with_ngrok
run_with_ngrok(app)

@app.route("/<filename>", methods=["POST","GET"])
def post_file(filename):
    with open(filename, "wb") as fp:
        fp.write(request.data)
    colorize(filename)
    return send_file(filename, mimetype='image/jpeg')

app.run()

"""
import threading
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':80}).start()
"""

172.28.0.2
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://662c7deeed6a.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/May/2021 12:47:11] "GET /img.jpeg HTTP/1.1" 200 -
